<a href="https://colab.research.google.com/github/fourfeatherz/DS2002S25/blob/main/python/superhero_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Superhero Database Assignment (SQLite & Google Colab)

## Objective
In this assignment, you will design and implement a **Superhero Database** using SQLite in Google Colab. This database will store superheroes, villains, and their battles. You will write SQL queries to analyze battle outcomes and relationships between heroes and villains.

### Group Size:
- 2 to 3 students per group.

### Tools Required:
- **Google Colab**
- **SQLite (built into Python)**
- **CSV files (provided for import)**

## Step 1: Setting Up Your SQLite Database

### Tasks:
1. Initialize SQLite in Google Colab.
2. Create a database named `superhero_db.sqlite`.
3. Define the database schema, ensuring it includes the following tables:
   - `heroes`: Stores superhero names, powers, and teams.
   - `villains`: Stores villains and their evil plans.
   - `battles`: Tracks battles between heroes and villains, including the outcome.
4. Implement primary keys and foreign keys where appropriate.

✅ **Checkpoint:** Ensure that the database and tables have been successfully created.

### Database Schema

#### Heroes:

* INT Primary Key Autoincrement
* STR Name Not Null Unique
* INT Foreign Key Powers

#### Powers:

* INT Primary Key Autoincrement
* STR Power Not Null Unique

#### Villain:

* INT Primary Key Autoincrement
* STR Name Not Null Unique
* INT Foreign Key Powers

#### Battles:
* INT Primary Key Autoincrement
* INT Foreign Key Heroes
* INT Foreign Key Villain
* ENUM -1,0,1 (LOSE,TIE,WIN)

Scratch that I thought we were suppose to design our own

In [19]:
import sqlite3

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# heroes table
cursor.execute('''
CREATE TABLE IF NOT EXISTS heroes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL UNIQUE,
    power TEXT NOT NULL,
    team TEXT NOT NULL
)
''')

# villain table
cursor.execute('''
CREATE TABLE IF NOT EXISTS villains (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL UNIQUE,
    evil_plan TEXT NOT NULL
)
''')

# battles table
cursor.execute('''
CREATE TABLE IF NOT EXISTS battles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    hero_id INTEGER,
    villain_id INTEGER,
    outcome TEXT NOT NULL
    )
''')

In [20]:
cursor.execute('''
    SELECT 
        name
    FROM 
        sqlite_schema
    WHERE 
        type ='table' AND 
        name NOT LIKE 'sqlite_%';
''')

for row in cursor.fetchall():
    print(row)

('heroes',)
('villains',)
('battles',)


## Step 2: Import Data from CSV Files

### Tasks:
1. Download the provided CSV files.
2. Upload them to Google Colab.
3. Write a script to import data from these files into the appropriate tables.
4. Verify that the data has been successfully inserted.

✅ **Checkpoint:** Ensure that each table contains the expected data from the CSV files.

In [21]:
import pandas as pd

heroes_df = pd.read_csv("heroes.csv")
villains_df = pd.read_csv("villains.csv")
battles_df = pd.read_csv("battles.csv")

heroes_df = heroes_df.drop(columns=["id"])
villains_df = villains_df.drop(columns=["id"])
battles_df = battles_df.drop(columns=["id"])

cursor.executemany('''
    INSERT INTO heroes (name, power, team)
    VALUES (?, ?, ?)
''', heroes_df.values)

cursor.executemany('''
    INSERT INTO villains (name, evil_plan)
    VALUES (?, ?)
''', villains_df.values)

cursor.executemany('''
    INSERT INTO battles (hero_id, villain_id, outcome)
    VALUES (?, ?, ?)
''', battles_df.values)

In [22]:
# Verify the data
cursor.execute('''
               SELECT COUNT(*) FROM heroes
                ''')
print(cursor.fetchone())
cursor.execute('''
                SELECT COUNT(*) FROM villains
                 ''')
print(cursor.fetchone())

cursor.execute('''
                SELECT COUNT(*) FROM battles
                 ''')
print(cursor.fetchone())


(10,)
(10,)
(30,)


## Step 3: Querying the Database

### Tasks:
Write SQL queries to answer the following:
1. Retrieve all superheroes and their powers.
2. Find all battles where the hero won.
3. List all villains and their evil plans.
4. Determine which superhero has fought the most battles.

✅ **Checkpoint:** Ensure that all queries return meaningful results from the database.

In [ ]:
cursor.execute('''
                SELECT * FROM heroes
                 ''')
print(cursor.fetchall())

cursor.execute('''
               SELECT * FROM heroes LEFT JOIN battles ON heroes.id = battles.hero_id WHERE battles.outcome = 'Hero Wins'
                ''')
print(cursor.fetchall())

cursor.execute('''
               SELECT * FROM villains
               ''')
print(cursor.fetchall())

cursor.execute('''
    SELECT heroes.name, COUNT(battles.id) AS win_count FROM heroes LEFT JOIN battles ON heroes.id = battles.hero_id
    WHERE battles.outcome = 'Hero Wins' GROUP BY heroes.name ORDER BY win_count DESC
''')
print(cursor.fetchone())

[(1, 'Spider-Man', 'Web-slinging', 'Avengers'), (2, 'Iron Man', 'Powered Armor', 'Avengers'), (3, 'Batman', 'Genius Detective', 'Justice League'), (4, 'Superman', 'Super Strength', 'Justice League'), (5, 'Thor', 'God of Thunder', 'Avengers'), (6, 'Wonder Woman', 'Amazonian Strength', 'Justice League'), (7, 'Black Panther', 'Enhanced Agility', 'Avengers'), (8, 'The Flash', 'Super Speed', 'Justice League'), (9, 'Hulk', 'Super Strength', 'Avengers'), (10, 'Doctor Strange', 'Mystic Arts', 'Avengers')]
[(9, 'Hulk', 'Super Strength', 'Avengers', 2, 9, 3, 'Hero Wins'), (1, 'Spider-Man', 'Web-slinging', 'Avengers', 5, 1, 7, 'Hero Wins'), (9, 'Hulk', 'Super Strength', 'Avengers', 7, 9, 8, 'Hero Wins'), (3, 'Batman', 'Genius Detective', 'Justice League', 8, 3, 2, 'Hero Wins'), (7, 'Black Panther', 'Enhanced Agility', 'Avengers', 9, 7, 5, 'Hero Wins'), (8, 'The Flash', 'Super Speed', 'Justice League', 13, 8, 9, 'Hero Wins'), (3, 'Batman', 'Genius Detective', 'Justice League', 20, 3, 4, 'Hero Wins

## Step 4: Add a Custom Feature

### Tasks:
1. Design and implement an additional table related to superheroes or villains.
2. Populate it with data.
3. Write at least one query to retrieve meaningful insights from the new table.

**Examples of Custom Features:**
- Sidekicks for superheroes
- Villain hideouts
- Battle locations
- Superhero equipment

✅ **Checkpoint:** Ensure that the new table and queries function correctly.

In [29]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS origins (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    origin TEXT NOT NULL
)
''')

origins_df = pd.read_csv("origins.csv")
origins_df = origins_df.drop(columns=["id"])

cursor.executemany('''
    INSERT INTO origins (name, origin)
    VALUES (?, ?)
''', origins_df.values)

cursor.execute('''
SELECT * FROM origins LEFT JOIN heroes ON origins.name = heroes.name LEFT JOIN villains ON origins.name = villains.name WHERE origins.origin = 'New York City'
''')
for row in cursor.fetchall():
    print(row)

(1, 'Spider-Man', 'New York City', 1, 'Spider-Man', 'Web-slinging', 'Avengers', None, None, None)
(2, 'Iron Man', 'New York City', 2, 'Iron Man', 'Powered Armor', 'Avengers', None, None, None)
(10, 'Doctor Strange', 'New York City', 10, 'Doctor Strange', 'Mystic Arts', 'Avengers', None, None, None)
(11, 'Green Goblin', 'New York City', None, None, None, None, 1, 'Green Goblin', 'Terrorize New York')
(21, 'Spider-Man', 'New York City', 1, 'Spider-Man', 'Web-slinging', 'Avengers', None, None, None)
(22, 'Iron Man', 'New York City', 2, 'Iron Man', 'Powered Armor', 'Avengers', None, None, None)
(30, 'Doctor Strange', 'New York City', 10, 'Doctor Strange', 'Mystic Arts', 'Avengers', None, None, None)
(31, 'Green Goblin', 'New York City', None, None, None, None, 1, 'Green Goblin', 'Terrorize New York')


## Bonus Challenge (+5 Points)

### Tasks:
1. Enhance your project by adding a visual or interactive component.

**Options:**
- Visualize battles using Matplotlib (e.g., a bar chart showing how many battles each superhero has fought).
- Make it interactive (e.g., allow users to input a superhero name and display their battles).

✅ **Checkpoint:** If attempting the bonus, ensure that your visualization or interactive component works correctly.

In [33]:
name = input("Enter the name of the hero: ")

cursor.execute('''
SELECT * FROM heroes LEFT JOIN battles ON battles.hero_id = heroes.id WHERE heroes.name = ?
''', (name,))

print(cursor.fetchall())

[(2, 'Iron Man', 'Powered Armor', 'Avengers', 6, 2, 8, 'Villain Wins'), (2, 'Iron Man', 'Powered Armor', 'Avengers', 12, 2, 1, 'Draw'), (2, 'Iron Man', 'Powered Armor', 'Avengers', 25, 2, 5, 'Draw')]


## Submission Guidelines

Each group must submit a **Google Colab Notebook (.ipynb)** containing:
- ✅ **Database schema (DDL)**
- ✅ **CSV imports & data insertion (DML)**
- ✅ **SQL queries & results**
- ✅ **Custom feature (bonus if applicable)**

## Grading Criteria (Total: 25 Points)

| Section                 | Points |
|-------------------------|--------|
| Database Setup          | 5      |
| Data Importing         | 10     |
| Querying the Data      | 10     |
| Extra Feature          | 5      |
| Bonus Challenge (Optional) | +5  |

### Final Notes:
- Be **creative** when designing superheroes and villains.
- **Comment your code** to explain its functionality.
- **Have fun** building your superhero universe!